# w261 Final Project - Clickthrough Rate Prediction


[Your team number (from the spreadsheet)]   
[Your team names]   
Summer 2019, section [Your section numbers>]   

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

# __Section 1__ - Question Formulation

# __Section 2__ - Algorithm Explanation

# __Section 3__ - EDA & Challenges

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts

In [1]:
#Initial EDA with 100K lines
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt 

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
sqlContext = SQLContext(sc)

In [5]:
#df = spark.read.option("header", "false").csv("data/train_small.txt")
df = sqlContext.read.format("csv") \
               .option("inferSchema", "true") \
               .option("header", "false") \
               .option("delimiter", "\t")\
               .load("data/train_small.txt")

In [6]:
#df = df.select(col("_c0").alias("CTR"))
df = df.withColumnRenamed("_c0", "CTR") \
       .withColumnRenamed("_c1", "NumVar1") \
       .withColumnRenamed("_c2", "NumVar2") \
       .withColumnRenamed("_c3", "NumVar3") \
       .withColumnRenamed("_c4", "NumVar4") \
       .withColumnRenamed("_c5", "NumVar5") \
       .withColumnRenamed("_c6", "NumVar6") \
       .withColumnRenamed("_c7", "NumVar7") \
       .withColumnRenamed("_c8", "NumVar8") \
       .withColumnRenamed("_c9", "NumVar9") \
       .withColumnRenamed("_c10", "NumVar10") \
       .withColumnRenamed("_c11", "NumVar11") \
       .withColumnRenamed("_c12", "NumVar12") \
       .withColumnRenamed("_c13", "NumVar13") 

df.select("CTR", "NumVar1", "NumVar2", "NumVar3", "NumVar4", "NumVar5", "NumVar6", "NumVar7", "NumVar8", "NumVar9", "NumVar10", "NumVar11", "NumVar12", "NumVar13").show()
#df.select("_c14", "_c15", "_c16", "_c17", "_c18", "_c19").show()
#data.select(col("Name").alias("name"), col("askdaosdka").alias("age"))

+---+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+
|CTR|NumVar1|NumVar2|NumVar3|NumVar4|NumVar5|NumVar6|NumVar7|NumVar8|NumVar9|NumVar10|NumVar11|NumVar12|NumVar13|
+---+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+
|  0|      1|      1|      5|      0|   1382|      4|     15|      2|    181|       1|       2|    null|       2|
|  0|      2|      0|     44|      1|    102|      8|      2|      2|      4|       1|       1|    null|       4|
|  0|      2|      0|      1|     14|    767|     89|      4|      2|    245|       1|       3|       3|      45|
|  0|   null|    893|   null|   null|   4392|   null|      0|      0|      0|    null|       0|    null|    null|
|  0|      3|     -1|   null|      0|      2|      0|      3|      0|      0|       1|       1|    null|       0|
|  0|   null|     -1|   null|   null|  12824|   null|      0|      0|      6|    null|  

In [7]:
df.select("_c14", "_c15", "_c16", "_c17", "_c18", "_c19", "_c20", "_c21").show()

+--------+--------+--------+--------+--------+--------+--------+--------+
|    _c14|    _c15|    _c16|    _c17|    _c18|    _c19|    _c20|    _c21|
+--------+--------+--------+--------+--------+--------+--------+--------+
|68fd1e64|80e26c9b|fb936136|7b4723c4|25c83c98|7e0ccccf|de7995b8|1f89b562|
|68fd1e64|f0cf0024|6f67f7e5|41274cd7|25c83c98|fe6b92e5|922afcc0|0b153874|
|287e684f|0a519c5c|02cf9876|c18be181|25c83c98|7e0ccccf|c78204a1|0b153874|
|68fd1e64|2c16a946|a9a87e68|2e17d6f6|25c83c98|fe6b92e5|2e8a689b|0b153874|
|8cf07265|ae46a29d|c81688bb|f922efad|25c83c98|13718bbd|ad9fa255|0b153874|
|05db9164|6c9c9cf3|2730ec9c|5400db8b|43b19349|6f6d9be8|53b5f978|0b153874|
|439a44a4|ad4527a2|c02372d0|d34ebbaa|43b19349|fe6b92e5|4bc6ffea|0b153874|
|68fd1e64|2c16a946|503b9dbc|e4dbea90|f3474129|13718bbd|38eb9cf4|1f89b562|
|05db9164|d833535f|d032c263|c18be181|25c83c98|7e0ccccf|d5b6acf2|0b153874|
|05db9164|510b40a5|d03e7c24|eb1fd928|25c83c98|    null|52283d1c|0b153874|
|05db9164|0468d672|7ae80d0f|80d8555a|2

In [8]:
!head -10 data/train_small.txt

0	1	1	5	0	1382	4	15	2	181	1	2		2	68fd1e64	80e26c9b	fb936136	7b4723c4	25c83c98	7e0ccccf	de7995b8	1f89b562	a73ee510	a8cd5504	b2cb9c98	37c9c164	2824a5f6	1adce6ef	8ba8b39a	891b62e7	e5ba7672	f54016b9	21ddcdc9	b1252a9d	07b5194c		3a171ecb	c5c50484	e8b83407	9727dd16
0	2	0	44	1	102	8	2	2	4	1	1		4	68fd1e64	f0cf0024	6f67f7e5	41274cd7	25c83c98	fe6b92e5	922afcc0	0b153874	a73ee510	2b53e5fb	4f1b46f3	623049e6	d7020589	b28479f6	e6c5b5cd	c92f3b61	07c540c4	b04e4670	21ddcdc9	5840adea	60f6221e		3a171ecb	43f13e8b	e8b83407	731c3655
0	2	0	1	14	767	89	4	2	245	1	3	3	45	287e684f	0a519c5c	02cf9876	c18be181	25c83c98	7e0ccccf	c78204a1	0b153874	a73ee510	3b08e48b	5f5e6091	8fe001f4	aa655a2f	07d13a8f	6dc710ed	36103458	8efede7f	3412118d			e587c466	ad3062eb	3a171ecb	3b183c5c		
0		893			4392		0	0	0		0			68fd1e64	2c16a946	a9a87e68	2e17d6f6	25c83c98	fe6b92e5	2e8a689b	0b153874	a73ee510	efea433b	e51ddf94	a30567ca	3516f6e6	07d13a8f	18231224	52b8680f	1e88c74f	74ef3502			6b3a5ca6		3a171ecb	9117a34a		
0	3	-1		0	2	0	3	0	0	1	1		0	8

## EDA related challenges

In [45]:
#CTR is not distributed uniformly between 0 and 1
df.filter(col("CTR") == 1).count()

22663

In [40]:
df.filter(col("CTR") == 0).count()

77337

## Logistic Regression Thoughts
Looking at paper in https://www.dropbox.com/s/s4x7wp8gjsh021d/TISTRespPredAds-Chappelle-CTR-Prediction-2014Paper.pdf?dl+=0

It looks like what we can do is this:

Step 1.1: Find a hashed feature vector for all the categorical variables 

> Question: What about numerical variables?

> Question: Do you hash each column separately?

Step 1.2: Create one-hot vectors for these categorical variables (does this make sense?)

Step 2.0: Take care of numerical variables so that they are also mapped into ranges?

Step 3.0: Run baseline logistic regression without any quadratic terms

Step 4.0: Run logistic regression including quadratic terms

In [60]:
#Feature hashing example
from pyspark.ml.feature import FeatureHasher

dataset = spark.createDataFrame([
    (2.2, True, "1", "foo"),
    (3.3, False, "2", "bar"),
    (4.4, False, "3", "baz"),
    (5.5, False, "4", "foo")
], ["real", "bool", "stringNum", "string"])

hasher = FeatureHasher(inputCols=["real", "bool", "stringNum", "string"],
                       outputCol="features")

featurized = hasher.transform(dataset)
featurized.select("features").show(truncate=False)

+--------------------------------------------------------+
|features                                                |
+--------------------------------------------------------+
|(262144,[174475,247670,257907,262126],[2.2,1.0,1.0,1.0])|
|(262144,[70644,89673,173866,174475],[1.0,1.0,1.0,3.3])  |
|(262144,[22406,70644,174475,187923],[1.0,1.0,4.4,1.0])  |
|(262144,[70644,101499,174475,257907],[1.0,1.0,5.5,1.0]) |
+--------------------------------------------------------+



In [63]:
#Let me try this on some column of the dataset
dataset = df.select("_c14", "_c15", "_c16")

In [140]:
hasher = FeatureHasher(numFeatures=8,inputCols=["_c15"],outputCol="c15_out")
featurized = hasher.transform(dataset)
featurized.printSchema()
featurized.select(featurized.c15_out).show()

root
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- c15_out: vector (nullable = true)

+-------------+
|      c15_out|
+-------------+
|(8,[0],[1.0])|
|(8,[2],[1.0])|
|(8,[5],[1.0])|
|(8,[7],[1.0])|
|(8,[5],[1.0])|
|(8,[3],[1.0])|
|(8,[6],[1.0])|
|(8,[7],[1.0])|
|(8,[3],[1.0])|
|(8,[3],[1.0])|
|(8,[5],[1.0])|
|(8,[0],[1.0])|
|(8,[1],[1.0])|
|(8,[5],[1.0])|
|(8,[0],[1.0])|
|(8,[0],[1.0])|
|(8,[3],[1.0])|
|(8,[3],[1.0])|
|(8,[2],[1.0])|
|(8,[5],[1.0])|
+-------------+
only showing top 20 rows



In [93]:
!pip install murmurhash3

  Running setup.py bdist_wheel for murmurhash3 ... done
  Stored in directory: /root/.cache/pip/wheels/06/eb/e4/f57324cd9c1bf001c9ba6d6926ad5231eca80964ed80b3610e
Successfully built murmurhash3
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [94]:
import mmh3

In [95]:
mmh3.hash('foo')

-156908512

In [103]:
mmh3.hash('foo', 42) % 16

14

In [106]:
mmh3.hash('foo')

-156908512